In [1]:
from sdRDM import DataModel
from sdrdm_database import DBConnector

In [2]:
# Establish a connection to the database
db = DBConnector(
    username="root",
    password="root",
    host="localhost",
    db_name="db",
    port=3306,
    dbtype="mysql",
)

In [3]:
# Fetch the sdRDM schema
lib = DataModel.from_markdown("./model.md")
lib.Root.meta_tree()

Root
├── id
├── value
├── nested
│   └── 0
│       ├── id
│       ├── another_value
│       └── value
└── some_values


ClassNode(/Root, class_name=None, constants={}, id=None, module=None, outer_type=None)

In [4]:
# Create tables for the sdRDM schema
db.create_tables(
    model=lib.Root,
    markdown_path="./model.md",
)


🚀 Creating tables for data model Root
│
├── Model 'Root' already registered. Skipping.
├── Table 'Root'. Already exists in database. Skipping.
├── Table 'Root_some_values'. Already exists in database. Skipping.
├── Table 'Root_nested'. Already exists in database. Skipping.
│
╰── 🎉 Created all tables for data model Root



In [5]:
# We are now seeking a way to insert an instance of 
# the sdRDM schema into the database.
print(db.connection.table("__model_meta__"))
print(db.connection.table("Root"))
print(db.connection.table("Root_nested"))
print(db.connection.table("Root_some_values"))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┓
┃ table       ┃ specifications                             ┃ github_url ┃ commit_hash ┃ part_of ┃ obj_name ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━┩
│ !string     │ json                                       │ string     │ string      │ string  │ string   │
├─────────────┼────────────────────────────────────────────┼────────────┼─────────────┼─────────┼──────────┤
│ Root        │ {'objects': [...], 'enums': [...], ... +3} │ NULL       │ NULL        │ NULL    │ Root     │
│ Root_nested │ NULL                                       │ NULL       │ NULL        │ Root    │ Nested   │
└─────────────┴────────────────────────────────────────────┴────────────┴─────────────┴─────────┴──────────┘



┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ value   ┃ Root_id                              ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ float64 │ !string                              │
├─────────┼──────────────────────────────────────┤
│   100.0 │ 0b9272bf-2d7b-429c-ac5a-f1a4ebc79542 │
│    20.0 │ 0cd575e4-1a91-4472-b274-00203bd0b04e │
│   100.0 │ 0d9ced19-e750-4b08-865d-397e050fd62b │
│    20.0 │ 236c39e5-5ddd-42df-901c-b89baa8c6004 │
│    20.0 │ 287d3c2e-1ddc-4891-95d5-8200d66dcc59 │
│    20.0 │ 2ca93076-0a34-404d-9463-0251251cbcdc │
│   100.0 │ 36ddf86f-2447-4911-b95f-173e133b8e34 │
│   100.0 │ 36faf7ad-8906-440e-937e-b7419045ca91 │
│   100.0 │ 386814f6-b662-4195-96be-66cc3a81f2ac │
│   100.0 │ 3e78ff3c-256a-4ae5-9e8f-d78e081f0a5b │
│       … │ …                                    │
└─────────┴──────────────────────────────────────┘



┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ another_value     ┃ value   ┃ Root_nested_id                       ┃ Root_id                              ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string            │ float64 │ !string                              │ string                               │
├───────────────────┼─────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│ hi there          │     nan │ 01190ee1-d2ac-416a-9ece-7a8269b0aad4 │ 287d3c2e-1ddc-4891-95d5-8200d66dcc59 │
│ something else    │     nan │ 03d0f64a-c1b3-438e-946f-ce4e323b3bff │ 0d9ced19-e750-4b08-865d-397e050fd62b │
│ hi there          │     nan │ 0452031f-4d3e-4c8f-93d6-3b01ae170896 │ 44235673-9705-4788-a34c-81cbcc331d2c │
│ something         │     nan │ 06401b57-2b19-4f17-919c-61ba282f5931 │ 36faf7ad-8906-440e-937e-b7419045ca91 │
│ hi there

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ some_values ┃ Root_id                              ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !string     │ string                               │
├─────────────┼──────────────────────────────────────┤
│ Hello       │ 36ddf86f-2447-4911-b95f-173e133b8e34 │
│ World       │ 36ddf86f-2447-4911-b95f-173e133b8e34 │
│ Whats       │ f63706ee-285e-4cc2-96e3-d009efc92264 │
│ up          │ f63706ee-285e-4cc2-96e3-d009efc92264 │
│ Hello       │ 82f389f8-d530-42c4-991d-700e12c52395 │
│ World       │ 82f389f8-d530-42c4-991d-700e12c52395 │
│ Whats       │ 2ca93076-0a34-404d-9463-0251251cbcdc │
│ up          │ 2ca93076-0a34-404d-9463-0251251cbcdc │
│ Hello       │ c72eded9-b9eb-44ce-bcc0-7b0cd7eaa218 │
│ World       │ c72eded9-b9eb-44ce-bcc0-7b0cd7eaa218 │
│ …           │ …                                    │
└─────────────┴──────────────────────────────────────┘



In [6]:
# Create datasets using the API that has been directly generated from the DB
model = db.get_table_api("Root")

dataset1 = model(value=100.0, some_values=["Hello", "World"])
dataset1.add_to_nested(another_value="something")
dataset1.add_to_nested(another_value="something else")

print(dataset1)

Root
├── id = root0
├── value = 100.0
├── nested
│   ├── 0
│   │   └── Nested
│   │       ├── id = nested0
│   │       └── another_value = something
│   └── 1
│       └── Nested
│           ├── id = nested1
│           └── another_value = something else
└── some_values = [Hello, World, ...]



In [7]:
# And another one
dataset2 = model(value=20.0, some_values=["Whats", "up"])
dataset2.add_to_nested(another_value="hi there")
dataset2.add_to_nested(another_value="this is dataset 2")

print(dataset2)

Root
├── id = root1
├── value = 20.0
├── nested
│   ├── 0
│   │   └── Nested
│   │       ├── id = nested2
│   │       └── another_value = hi there
│   └── 1
│       └── Nested
│           ├── id = nested3
│           └── another_value = this is dataset 2
└── some_values = [Whats, up, ...]



In [8]:
# Insert the new dataset into the database
db.insert(dataset1, dataset2, verbose=True)

Added dataset Root (c9e02eb5-23e3-43df-8315-a39edfb6b29a)
Added dataset Root (cd23473d-5e05-4a52-b4cd-68cf266130a9)


In [9]:
# Lets see if it worked
print(db.connection.table("Root"))
print(db.connection.table("Root_nested"))
print(db.connection.table("Root_some_values"))

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ value   ┃ Root_id                              ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ float64 │ !string                              │
├─────────┼──────────────────────────────────────┤
│   100.0 │ 0b9272bf-2d7b-429c-ac5a-f1a4ebc79542 │
│    20.0 │ 0cd575e4-1a91-4472-b274-00203bd0b04e │
│   100.0 │ 0d9ced19-e750-4b08-865d-397e050fd62b │
│    20.0 │ 236c39e5-5ddd-42df-901c-b89baa8c6004 │
│    20.0 │ 287d3c2e-1ddc-4891-95d5-8200d66dcc59 │
│    20.0 │ 2ca93076-0a34-404d-9463-0251251cbcdc │
│   100.0 │ 36ddf86f-2447-4911-b95f-173e133b8e34 │
│   100.0 │ 36faf7ad-8906-440e-937e-b7419045ca91 │
│   100.0 │ 386814f6-b662-4195-96be-66cc3a81f2ac │
│   100.0 │ 3e78ff3c-256a-4ae5-9e8f-d78e081f0a5b │
│       … │ …                                    │
└─────────┴──────────────────────────────────────┘



┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ another_value     ┃ value   ┃ Root_nested_id                       ┃ Root_id                              ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string            │ float64 │ !string                              │ string                               │
├───────────────────┼─────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│ hi there          │     nan │ 01190ee1-d2ac-416a-9ece-7a8269b0aad4 │ 287d3c2e-1ddc-4891-95d5-8200d66dcc59 │
│ something else    │     nan │ 03d0f64a-c1b3-438e-946f-ce4e323b3bff │ 0d9ced19-e750-4b08-865d-397e050fd62b │
│ hi there          │     nan │ 0452031f-4d3e-4c8f-93d6-3b01ae170896 │ 44235673-9705-4788-a34c-81cbcc331d2c │
│ something         │     nan │ 06401b57-2b19-4f17-919c-61ba282f5931 │ 36faf7ad-8906-440e-937e-b7419045ca91 │
│ somethin

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ some_values ┃ Root_id                              ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !string     │ string                               │
├─────────────┼──────────────────────────────────────┤
│ Hello       │ 36ddf86f-2447-4911-b95f-173e133b8e34 │
│ World       │ 36ddf86f-2447-4911-b95f-173e133b8e34 │
│ Whats       │ f63706ee-285e-4cc2-96e3-d009efc92264 │
│ up          │ f63706ee-285e-4cc2-96e3-d009efc92264 │
│ Hello       │ 82f389f8-d530-42c4-991d-700e12c52395 │
│ World       │ 82f389f8-d530-42c4-991d-700e12c52395 │
│ Whats       │ 2ca93076-0a34-404d-9463-0251251cbcdc │
│ up          │ 2ca93076-0a34-404d-9463-0251251cbcdc │
│ Hello       │ c72eded9-b9eb-44ce-bcc0-7b0cd7eaa218 │
│ World       │ c72eded9-b9eb-44ce-bcc0-7b0cd7eaa218 │
│ …           │ …                                    │
└─────────────┴──────────────────────────────────────┘



In [10]:
# Finally, recover the data from the database by filtering
retrieved = db.get("Root")
print(retrieved[0])

Root
├── id = 0b9272bf-2d7b-429c-ac5a-f1a4ebc79542
├── value = 100.0
├── nested
│   ├── 0
│   │   └── Nested
│   │       ├── id = 48f81a12-fe2b-4ec3-8218-aefd0dca0a09
│   │       └── another_value = something
│   └── 1
│       └── Nested
│           ├── id = 810ee083-4a24-478c-bc94-1505575ca67d
│           └── another_value = something else
└── some_values = [Hello, World, ...]

